<center><a href="https://www.fluentd.org/" target="_blank"><img src="img/fluentd.png"></a><center>

## What is fluentd?

"*Fluentd is an open source data collector, which lets you unify the data collection and consumption for a better use and understanding of data.*"

## Why use fluentd?

- Unified Logging with JSON

- Pluggable Architecture

- Minimum Resources Required

- Built-in Reliability

## Unified Logging with JSON

"*Fluentd tries to structure data as JSON as much as possible: this allows Fluentd to unify all facets of processing log data: collecting, filtering, buffering, and outputting logs across multiple sources and destinations.*"

<table>
    <tr>
        <td><img src="img/before_fluentd.png" width="100%"></td>
        <td><img src="img/after_fluentd.png" width="100%"></td>
    </tr>
</table>

## Pluggable Architecture

"*Fluentd has a flexible plugin system that allows the community to extend its functionality. Our 500+ community-contributed plugins connect dozens of data sources and data outputs. By leveraging the plugins, you can start making better use of your logs right away.*"

## Minimum Resources Required

"*Fluentd is written in a combination of C language and Ruby, and requires very little system resource. The vanilla instance runs on 30-40MB of memory and can process 13,000 events/second/core. If you have tighter memory requirements (-450kb), check out Fluent Bit, the lightweight forwarder for Fluentd.*"

## Built-in Reliability

"*Fluentd supports memory- and file-based buffering to prevent inter-node data loss. Fluentd also supports robust failover and can be set up for high availability. 2,000+ data-driven companies rely on Fluentd to differentiate their products and services through a better use and understanding of their log data.*"

## Now the problem is...

<center><img src="img/logstash_or_fluentd.jpg" width="25%"></center>

## Fluentd vs. LogStash

<blockquote class="twitter-tweet"><p lang="en" dir="ltr">There are three important decisions in life:who you love,who you fight,and how you ship your logs <a href="https://t.co/Ow1CIDt7sI">https://t.co/Ow1CIDt7sI</a> <a href="https://twitter.com/hashtag/fluentd?src=hash&amp;ref_src=twsrc%5Etfw">#fluentd</a> <a href="https://twitter.com/hashtag/logstash?src=hash&amp;ref_src=twsrc%5Etfw">#logstash</a></p>&mdash; Luc Juggery (@lucjuggery) <a href="https://twitter.com/lucjuggery/status/893359554863316993?ref_src=twsrc%5Etfw">August 4, 2017</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script> 

### Popularity and Livelihood of the tool

**LogStash:** 7k stars on GitHub, IRC channel and a forum.

**Fluentd:** 4k stars on Github, Slack channel, newsletters and Google group.

### Endorsing company

**LogStash:** part of the ELK stack.

**Fluentd:** part of the CNCF (Cloud Native Computing Foundation).

### Platform Compatibility and Extensibility

**LogStash:** Windows and Linux. about 200 plugins on GitHub.

**Fluentd:** Windows and Linux. about 500 plugins on GitHub.

### Transport Ecosystem

**LogStash:** limited to an on-memory queue that holds 20 events.

**Fluentd:** highly configurable buffering system.

### Monitoring & Tuning

**LogStash:** Metrics filter, can be visualized in 3rd party such as Graphite.

**Fluentd:** built-in monitoring agent, plug-ins to integrate with your monitoring stack.

<center><img src="img/plug-ins.jpg" width="20%"></center>

### Configuration DSL

**LogStash:** routing into a single stream, then sends to the correct destination.

**Fluentd:** routing based on event tags.

## Fluentd example

Now we will create an "Hello world" example using the official <a href="https://hub.docker.com/r/fluent/fluentd/" target="_blank">Fluentd Docker container.</a> and a simple script Python.

### Script Python

```python
import threading
import time
import random
from fluent import sender
from fluent import event
```

```shell
$ pip install fluent-logger
```

```python
thread_name = {
    'one':      '{\"message\":\"Hello\",\"from\":\"thread.one\"}', 
    'two':      'Hello thread.two', 
    'three':      'Hello thread.three', 
    'four':    '{\"message\":\"Hello\",\"from\":\"thread.four\"}'
}
```

```python
if __name__ == "__main__":
    print("Starting threads")
    for name in thread_name:
        threading.Thread(target=generator, args=(name, )).start()
```

```python
def generator(name):
    time.sleep(5)
    sender.setup("thread", host='fluentd', port=9888)
    while(True):
        event.Event(name, {
            'log': thread_name[name]
        })
        time.sleep(random.randint(1,5))
```

### Configuration file

"*For a Docker container, the default location of the config file is `/fluentd/etc/fluent.conf`. To mount a config file from outside of Docker, use a `bind-mount`.*"

#### Source

Directives determine the input sources.

<pre>
&lt;source&gt;
  @type forward
  port 9888
  bind "0.0.0.0"
&lt;/source&gt;
</pre>

#### Filter

Directives determine the event processing pipelines.

##### thread.one and thread.four

<div style="float:left;width:50%;">
    <pre>
    &lt;filter thread.one&gt;
      @type parser
      key_name log
      hash_value_field parsed
      &lt;parse&gt;
        @type json
      &lt;/parse&gt;
    &lt;/filter&gt;
    </pre>
</div>
<div style="float:left;width:50%;">
    <pre>
    &lt;filter thread.four&gt;
      @type parser
      key_name log
      hash_value_field parsed
      &lt;parse&gt;
        @type json
      &lt;/parse&gt;
    &lt;/filter&gt;
    </pre>
</div>

##### thread.two and thread.three

<div style="float:left;width:50%;">
    <pre>
    &lt;filter thread.two&gt;
      @type parser
      key_name log
      hash_value_field parsed
      &lt;parse&gt;
        @type regexp
        expression /^(?&lt;message&gt;[^ ]*) (?&lt;from&gt;[^ ]*)&#36;/
      &lt;/parse&gt;
    &lt;/filter&gt;
    </pre>
</div>
<div style="float:left;width:50%;">
    <pre>
    &lt;filter thread.four&gt;
      @type parser
      key_name log
      hash_value_field parsed
      &lt;parse&gt;
        @type regexp
        expression /^(?&lt;message&gt;[^ ]*) (?&lt;from&gt;[^ ]*)&#36;/
      &lt;/parse&gt;
    &lt;/filter&gt;
    </pre>
</div>

#### Match

Directives determine the output destinations.

<div style="float:left;width:50%;">
    <pre>
    &lt;match thread.one&gt;
      @type stdout
    &lt;/match&gt;
    </pre>
</div>
<div style="float:left;width:50%;">
    <pre>
    &lt;match thread.two&gt;
      @type stdout
    &lt;/match&gt;
    </pre>
</div>
<div style="float:left;width:50%;">
    <pre>
    &lt;match thread.three&gt;
      @type stdout
    &lt;/match&gt;
    </pre>
</div>
<div style="float:left;width:50%;">
    <pre>
    &lt;match thread.four&gt;
      @type stdout
    &lt;/match&gt;
    </pre>
</div>

### docker-compose.yml

```docker
fluentd:
        image: fluent/fluentd:edge-debian
        container_name: fluentd
        hostname: fluentd
        ports: 
            - "9888:9888"
        volumes: 
            - ./conf:/fluentd/etc
        networks: 
            tap:
                ipv4_address: 10.0.100.2
```

```docker
generator:
        image: python:3.9.5
        container_name: generator
        hostname: generator
        depends_on: 
            - fluentd
        volumes: 
            - ./generator:/generator
        command: "bash -c 'pip install fluent-logger && \
                  python /generator/app.py'"
        networks: 
            tap:
                ipv4_address: 10.0.100.3
```

<center><img src="img/end.jpg"></center>